In [75]:
import os
import glob
import pandas as pd
import numpy as np

In [76]:
# 合并CSV
# source: https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

'''
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv("INI_Holder_Detail_Combined.csv", index=False, encoding='utf-8-sig')
'''

'\nextension = \'csv\'\nall_filenames = [i for i in glob.glob(\'*.{}\'.format(extension))]\n\ncombined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])\ncombined_csv.to_csv("INI_Holder_Detail_Combined.csv", index=False, encoding=\'utf-8-sig\')\n'

In [77]:
df = pd.read_csv('INI_Holder_Detail_Combined.csv')
df_names = pd.read_csv('合资金融机构.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
names = df_names[['机构简称']]
names['count'] = 0
names

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,机构简称,count
0,高盛高华,0
1,摩根士丹利华鑫,0
2,瑞银,0
3,瑞信,0
4,中德,0
...,...,...
77,首奕,0
78,韩华,0
79,润晖,0
80,荷宝,0


In [79]:
len(df)

5281899

In [80]:
# calculate levenshtein ratio & distance
# source: https://www.datacamp.com/community/tutorials/fuzzy-string-python
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [86]:
df_test = df.loc[0:100000]

In [82]:
for i in range(len(names)):
    name = names.iloc[i]['机构简称']
    print(name, i)
    count = (df_test['ShareHolderName'].apply(lambda x: levenshtein_ratio_and_distance(name, x, True)) > 0.2).sum()
    names.at[i,'count'] = count
    print(count)

高盛高华 0
14
摩根士丹利华鑫 1
724
瑞银 2
150
瑞信 3
549
中德 4
44
申港 5
8
华兴 6
253
汇丰前海 7
130
东亚前海 8
126
野村东方 9
454
摩根大通 10
126
金圆统一 11
566
大和 12
61
星展 13
3
国泰 14
680
鹏华 15
853
嘉实 16
683
长盛 17
527
富国 18
720
融通 19
365
国投瑞银 20
2296
泰达宏利 21
951
华宝 22
792
国联安 23
551
海富通 24
1027
景顺长城 25
1675
兴证全球 26
1053
申万菱信 27
729
中海 28
743
上投摩根 29
2147
光大保德信 30
738
国海富兰克林 31
774
华泰柏瑞 32
1760
工银瑞信 33
1824
交银施罗德 34
587
中信保诚 35
449
建信 36
553
汇丰晋信 37
333
中邮创业 38
546
信达澳银 39
310
中欧 40
184
浦银安盛 41
675
农银汇理 42
2832
民生加银 43
498
方正富邦 44
196
华宸未来 45
80
华润元大 46
166
中加 47
86
国开泰富 48
549
永赢 49
57
圆信永丰 50
79
恒生前海 51
78
富达利泰 52
162
瑞银资产 53
4977
英仕曼 54
0
富敦 55
7
惠理 56
19
景顺纵横 57
42
路博迈 58
18
安本 59
81
施罗德 60
455
贝莱德 61
4
安中 62
163
桥水 63
2
元胜 64
1
毕盛 65
0
瀚亚 66
16
未来益财 67
20
联博汇智 68
74
安联 69
74
德劭 70
3
罗素 71
0
弘收 72
0
威廉欧奈尔 73
3
鲍尔赛嘉 74
2
迈德瑞 75
4
柏基 76
539
首奕 77
0
韩华 78
62
润晖 79
0
荷宝 80
5
澳帝桦 81
0


In [88]:
names = "瑞信"

count = 0
for i in range(len(df_test)):
    temp = levenshtein_ratio_and_distance(names, str(df_test.iloc[i]['ShareHolderName']),ratio_calc = True)
    
    if temp > 0.2:
        print('index: ' + str(i))
        print('levenshtein ratio: ' + str(temp))
        print(df_test.iloc[i])
        count += 1
print(count)


index: 140
levenshtein ratio: 0.3333333333333333
Symbol                          600775
EndDate                     2005-03-31
SystematicsID                    P9410
Systematics                     其他机构持股
ShareHolderName                   信达投资
CategoryCode                     P2239
Holdshares                      157300
HoldProportion                   0.024
Price                             3.83
LastEndDate                        NaN
LastShareHolderID                  NaN
LastHoldshares                     NaN
LastHoldProportion                 NaN
LastPrice                          NaN
RightsRatio                          1
Status                               3
IndustryCode                       C39
IndustryName          计算机、通信和其他电子设备制造业
Source                               2
ShareHolderID              5.05824e+08
Name: 140, dtype: object
index: 150
levenshtein ratio: 0.3333333333333333
Symbol                          600775
EndDate                     2005-09-30
SystematicsID      